### Step1 导入相关包

In [34]:
import pandas as pd
import numpy as np
import faiss
from llama_index.core import Settings
from llama_index.core import  SimpleKeywordTableIndex,Document, Prompt
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core import VectorStoreIndex
from transformers import AutoModel, AutoTokenizer



### Step2 加载外部知识

In [4]:
#  从Excel文件导入外部知识库
def load_knowledge_base_from_excel(file_path):
    data = pd.read_excel('./电力现货市场101问.xlsx',header=None)
    questions = data.loc[1:,0]
    answers = data.loc[1:,1]

    QA_list = []
    for item in zip(questions,answers):
        qa = item[0] + '\n' + item[1]
        QA_list.append(qa)
    documents = []
    for i in range(0,len(QA_list)):
        qa_doc = Document(text=QA_list[i], metadata={"source": "PowerMarketQA\电力现货市场101问.xlsx","order":i+1})
        documents.append(qa_doc)
    return documents

datas = load_knowledge_base_from_excel('./电力现货市场101问.xlsx')
datas[0]

Document(id_='1855af09-f634-4c1b-99e9-b10daa41b577', embedding=None, metadata={'source': 'PowerMarketQA\\电力现货市场101问.xlsx', 'order': 1}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='1.什么是电力市场？电力市场与普通商品市场有哪些差异？电力市场有哪些特征？\n（1）电力市场的概念。我国关于电力市场的权威解释始见于《中国电力百科全书电力系统卷（第二版）》。电力市场的定义为：基于市场经济原则，电力市场的定义为基于市场经济原则，为实现电力商品交换的电力工业组织结构、经营管理和运行规则的总和。电力市场又是一个具体的执行系统，包括交易场所、交易管理系统、计量和结算系统、信息和通信系统等。上面从组织和实操两个维度对电力市场进行了描绘。通常电力市场包括广义和狭义两种含义。比照商品市场的一般定义，广义的电力市场泛指电力流通交换的领域。按照该含义，自电力作为商品实现交换之日起，电力市场就已经存在。当今在涉及市场规模和范围的语境下，运用的便是广义电力市场的概念。广义的电力市场有着明确的地域和容量指向，电力市场的地理边界可能差异很大，例如单一州（省）域内的电力市场，或者整个国家的电力市场，乃至跨国的电力市场。而这种地理边界受限于电网的覆盖范围，由于电网是电力传输的唯一通道，故多大范围的电网才可能有多大范围的电力市场；电力市场容量也同样存在很大的差异。狭义的电力市场是指现代竞争性的电力市场，《中国电力百科全书》定义的电力市场即为狭义的电力市场，建立电力市场旨在通过开放、竞争等市场手段实现电力能源资源的优化配置。所谓基于市场经济原则主要是指电能生产者和使用者本着公平竞争、自愿互利的原则，通过协商、竞价等方式，就电能及其相关产品进行交易，通过市场竞争确定价格和数量的市场原则。目前广泛讨论的电力市场，除特殊语境下，通常都是指狭义的电力市场。之所以称之为狭义的电力市场，主要原因有二。1）专指现代电力市场，狭义的电力市场兴起于20世纪80年代，是在电力作为商品出现的百年之后，而这百年间，广义电力市场的形态发生了曲折的变迁。电力工业初创期的

### Step3 加载embedding模型

In [5]:
# 2. 加载 Hugging Face 上的 M3E embedding 模型
def get_m3e_embedding_model(model_name='moka-ai/m3e-base'):
    # tokenizer = AutoTokenizer.from_pretrained(model_name)
    # model = AutoModel.from_pretrained(model_name)
    # embedding_model = HuggingFaceEmbedding(tokenizer=tokenizer, model=model)

    # loads moka-ai/m3e-base
    embedding_model = HuggingFaceEmbedding(model_name="moka-ai/m3e-base")
    return embedding_model

embedding_model = get_m3e_embedding_model()

### Step4 向量索引

In [41]:
# 3. 使用 Faiss 向量数据库索引文档
def build_faiss_index(documents, embedding_model):
    # 获取所有文档的嵌入
    embeddings = [embedding_model.get_text_embedding(doc.text) for doc in documents]
    # print(embeddings[0])
    # 使用faiss构建向量索引
    dim = len(embeddings[0])
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings))
    # Settings.embed_model = embedding_model
    # index  =  VectorStoreIndex.from_documents( documents,show_progress= True)
    # 创建 Faiss 向量存储对象
    # vector_store = FaissVectorStore(faiss_index=index)
    # vector_store.add(embeddings)
    return index

vector = build_faiss_index(datas,embedding_model)
vector

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000002156821E570> >

### Step5 构造prompt

In [14]:
# 4. 构造专属的 Prompt
def construct_prompt(query, context):
    prompt_template = (
        "给出以下查询：\nQuery: {query}\n\n相关文档内容如下：\n{context}\n\n请基于上述信息回答查询。"
    )
    return prompt_template.format(query=query, context=context)



### Step6 加载LLM

In [16]:
# 5. 使用百川大模型回答问题
from zhipuai_llm import ZhipuLLM
def answer_query_with_baichuan(prompt):
    zhipu_llm = ZhipuLLM()
    response = zhipu_llm.complete(prompt)
    return response['text']



d:\Download\miniconda3\envs\py3.9\lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in ZhipuLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


### Step7 构建RAG流程

In [45]:
query = "电力市场与普通商品市场的差异？"
query_vector = embedding_model.get_text_embedding(query) 
D,I = vector.search(np.array(query_vector), 3) 
print(I[:5]) 

ValueError: not enough values to unpack (expected 2, got 1)

In [21]:
# 主流程
def main(query, excel_path='./电力现货市场101问.xlsx', embedding_model_name='moka-ai/m3e-base'):
    # 导入知识库
    documents = load_knowledge_base_from_excel(excel_path)
    
    # 获取嵌入模型
    embedding_model = get_m3e_embedding_model(embedding_model_name)
    
    # 构建 Faiss 索引
    vector_store = build_faiss_index(documents, embedding_model)
    
    # 查询并获取相关文档
    results = vector_store.query([query],similarity_top_k=3)
    context = "\n".join([doc.text for doc in results])
    
    # 构造 Prompt 并回答
    prompt = construct_prompt(query, context)
    answer = answer_query_with_baichuan(prompt)
    
    print("Query:", query)
    print("Answer:", answer)



### Step8 运行示例

In [20]:
# 执行示例
if __name__ == "__main__":
    query = "电力市场与普通商品市场的差异？"
    main(query)

AttributeError: 'str' object has no attribute 'filters'